# SQL

Для анализа дан небольшой фаил размеров в 240 Мб содержащий только информацию о новостройках. Содержит срезы на каждый день(кроме дней поломок - простоя сервиса) Т.е. есть по каждому обекту есть информация за каждый день. Подучатеся в год(365 дней) информация об обект дублируется 365 раз.
Учтите что сложный селект может работать не быстро.

Для анализа использовать фаил с базой /home/synchronize_data/price_analytics/24.db.copied

In [1]:
!ls -la /home/synchronize_data/price_analytics/24.db.copied

-rw-r--r-- 1 synchronize_data synchronize_data 264585216 Jun 11 09:08 /home/synchronize_data/price_analytics/24.db.copied


# Пример использования: задача вывести самую дорогую квартиру из имеющихся.

Запустите следующие 6 строчек, они должны отработать корректно.

In [2]:
import matplotlib.pyplot as plt
import sqlite3
import json

In [3]:
connection = sqlite3.connect('/home/synchronize_data/price_analytics/24.db.copied')
cursor = connection.cursor()

Поля рекомендованные:
            id,
            "title",
            "price",
            "price_sale",
            date(datetime_create) as date,
            "brand",
            "apartment_area",
            "apartment_floor",
            "apartment_floors_total",
            "apartment_room",
            "apartment_ppm"

In [4]:
cursor.execute("""
SELECT 
    *
FROM copied
where 
    category = "Новостройки" 
    or category = "Реновация"
ORDER BY price desc
LIMIT 1
""")

In [5]:
flat = cursor.fetchall()

In [6]:
flat

[(44871689,
  'Саввинская 27 | 27 | 2 | 340.9 | Саввинская 27 | 55.730824,37.557959 | 1',
  847419076,
  847419076,
  '2024-06-02 06:13:37.130681',
  'Новостройки',
  'level.ru',
  'https://level.ru/old/sav27/apartment/6room/5-1/',
  340.9,
  2,
  6,
  6,
  2485829)]

In [7]:
%reload_ext sql 
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

In [8]:
%sql sqlite:////home/synchronize_data/price_analytics/24.db.copied
#%sql sqlite:///24.db.copied

In [9]:
#далее можно вот так писать sql запросы и даже передавать результаты их выполнения в переменные - смотрите по ссылке вверху этого ноутбука
%sql SELECT * from copied WHERE id<600

id,title,price,price_sale,datetime_create,category,brand,url,apartment_area,apartment_floor,apartment_floors_total,apartment_room,apartment_ppm
598,"Причальный Bосток 29 65.3 Причальный пр., д. 8 55.76018,37.517482",37527037,26268925,2022-12-05 13:38:36.641439,Новостройки,level.ru,url_example,None,None,None,None,None
599,"Саввинская 17 17 4 82.5 55.734600731695195,37.565715889000494",202359478,202359478,2022-12-05 13:38:36.652957,Новостройки,level.ru,url_example,None,None,None,None,None


# Задачи:

In [10]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'

name
copied


Выведите список новостроек в порядке убывания количества доступных к покупке квартир.

In [42]:
%%sql
SELECT brand, COUNT(DISTINCT title) as count
FROM copied
WHERE datetime_create like "2024-06-10%"
GROUP BY brand
ORDER BY count DESC
LIMIT 10

brand,count
gk-osnova.ru,2028
level.ru,1104
donstroy.moscow,1071


Найдите среднюю цену квадратного метра по всем новостройкам  

In [56]:
%%sql
SELECT ROUND(AVG(apartment_ppm)) as apartment_ppm_total
FROM copied
WHERE datetime_create like "2024-06-10%" and apartment_ppm > 0
LIMIT 10

apartment_ppm_total
386852.0


Выведите список новостроек в порядке убывания общего количества квартир.  

In [61]:
%%sql
SELECT brand, COUNT(DISTINCT title) as count
FROM copied
GROUP BY brand
ORDER BY count DESC
LIMIT 10

brand,count
level.ru,12459
donstroy.moscow,5920
gk-osnova.ru,2973
fr.mos.ru,2533
Capital Group,48
Галс-Девелопмент,2


Найдите среднюю площадь квартир в новостройках с количеством комнат больше 3  

In [65]:
%%sql
SELECT ROUND(AVG(apartment_ppm)) as apartment_ppm_total
FROM copied
WHERE datetime_create like "2024-06-10%" and apartment_ppm > 0 and apartment_room > 3
LIMIT 10

apartment_ppm_total
532762.0


Найдите среднюю цену квадратного метра по всем новостройкам на конкретный день.  

In [66]:
%%sql
SELECT ROUND(AVG(apartment_ppm)) as apartment_ppm_total
FROM copied
WHERE datetime_create like "2023-06-10%" and apartment_ppm > 0
LIMIT 10

apartment_ppm_total
441790.0


Найдите среднюю цену за квадратный метр в новостройках с количеством этажей больше 15.

In [67]:
%%sql
SELECT ROUND(AVG(apartment_ppm)) as apartment_ppm_total
FROM copied
WHERE datetime_create like "2024-06-10%" and apartment_floors_total > 15
LIMIT 10

apartment_ppm_total
453861.0


Найти все новостройки которые появились в 2023 году  

In [84]:
%%sql
SELECT DISTINCT title
FROM copied
WHERE datetime_create like "2023%" and title not in (
    SELECT title
    FROM copied
    WHERE datetime_create like "2022-12-31%"
)
LIMIT 10

title
"Павелецкая Сити 6 9 23.6 ул. Дубининская, д. 59-69 55.71962004653103,37.63727228350035"
"Стрешнево Восток 16 21.5 Волоколамское ш., д. 81, корп. 2 55.82495356307498,37.411886931513436"
"Академическая 1 9 25.4 Профсоюзная ул., д. 2/22 55.687477,37.572378"
"Причальный Юг 14 22.1 Причальный пр., д. 8 55.76018,37.517482"
"Причальный Bосток 25 65.4 Причальный пр., д. 8 55.76018,37.517482"
"Причальный Bосток 27 65.3 Причальный пр., д. 8 55.76018,37.517482"
"Нагатинская В 11 56.0 г. Москва, Нагатинская набережная, уч. 10А 55.682432,37.637749"
"Нагатинская D 9 45.2 г. Москва, Нагатинская набережная, уч. 10А 55.682432,37.637749"
"Павелецкая Сити 7 4 62.7 ул. Дубининская, д. 59-69 55.71962004653103,37.63727228350035"
"Павелецкая Сити 9 17 78.9 ул. Дубининская, д. 59-69 55.71962004653103,37.63727228350035"


Найти все новостройки которые имеют больше 50 квартир  

In [97]:
%%sql
SELECT brand, COUNT(DISTINCT title) as count
FROM copied
GROUP BY brand
HAVING count > 50
ORDER BY count DESC
LIMIT 10

brand,count
level.ru,12459
donstroy.moscow,5920
gk-osnova.ru,2973
fr.mos.ru,2533


Найдите максимальную и минимальную цену среди всех новостроек  

In [99]:
%%sql
SELECT MIN(price_sale), MAX(price_sale)
FROM copied
WHERE price_sale > 0
LIMIT 10

MIN(price_sale),MAX(price_sale)
551000,847419076


Найти все новостройки в которых есть квартиры площадью больше 100 кв метров  

In [101]:
%%sql
SELECT DISTINCT title, apartment_area
FROM copied
WHERE apartment_area > 100
LIMIT 10

title,apartment_area
"Саввинская 27 27 2 137.5 Саввинская 27 55.730669,37.557942",137.5
"Саввинская 27 27 3 139.5 Саввинская 27 55.730669,37.557942",139.5
"Саввинская 17 17 4 110.5 55.734600731695195,37.565715889000494",110.5
"Саввинская 27 27 6 189.9 Саввинская 27 55.730669,37.557942",189.9
"Мичуринский 6 15 112.3 Озерная ул., вл. 7 55.687726,37.471704",112.3
"Мичуринский 5 36 178.0 Озерная ул., вл. 7 55.687726,37.471704",178
"Мичуринский 5 36 245.5 Озерная ул., вл. 7 55.687726,37.471704",245.5
"Мичуринский 6 18 112.3 Озерная ул., вл. 7 55.687726,37.471704",112.3
"Мичуринский 5 36 185.9 Озерная ул., вл. 7 55.687726,37.471704",185.9
"Мичуринский 4 44 249.1 Озерная ул., вл. 7 55.687726,37.471704",249.1


Найти новостройки котрые были построекны застройщщиком Донстрой  

In [103]:
%%sql
SELECT DISTINCT title, brand
FROM copied
WHERE brand = 'donstroy.moscow'
LIMIT 10

title,brand
"Символ Независимость Корпус 19, 2 секц 4 27.1",donstroy.moscow
"Символ Независимость Корпус 19, 2 секц 3 28.3",donstroy.moscow
"Символ Независимость Корпус 19, 2 секц 3 29.9",donstroy.moscow
"Символ Независимость Корпус 19, 2 секц 5 27.1",donstroy.moscow
"Символ Независимость Корпус 19, 2 секц 5 29.4",donstroy.moscow
"Символ Независимость Корпус 19, 2 секц 5 28.3",donstroy.moscow
"Символ Независимость Корпус 19, 2 секц 4 29.9",donstroy.moscow
"Символ Независимость Корпус 19, 2 секц 5 29.9",donstroy.moscow
"Символ Независимость Корпус 19, 2 секц 5 31.4",donstroy.moscow
"Символ Независимость Корпус 19, 2 секц 11 35.9",donstroy.moscow


Найти всех застройщиков которые построили более 5 новостроек в 2023 году  

In [106]:
%%sql
SELECT brand, COUNT(DISTINCT title) as count
FROM copied
WHERE datetime_create like "2023%" and title not in (
    SELECT title
    FROM copied
    WHERE datetime_create like "2022-12-31%"
)
GROUP BY brand
HAVING count > 5
ORDER BY count DESC
LIMIT 10

brand,count
level.ru,5397
fr.mos.ru,2220
Capital Group,48


Вывести список жилых комплексов с наибольшим количеством проданных квартир за последние 3 месяца. Вывести количество проданных квартир и процент от общего количества квартир этого комплекса

# Задачи с обогащением:

Построить месячный график количества продаж новостроек за последний год.  

Выведите список новостроек в западном районе.  

Найти все новостройки в ЗАО и СЗАО которые имеют больше 10 этажей  

Найти самый популярный размер квартир в каждом округе Москвы, и посчитать количеств продаж каждого размера ( сейчас за размер принимаем количество комнат)  

Найти все новостройки в пределах заданного радиуса от опеределнного места. И определить расстояние от каждой новостройки до заданного места, распределив их по возрастанию расстояния до заданного места.  

Построить график средней стоимости квадратного метра по месяцам, начиная с q2 2023 по q2 2024  

Найти все новостройки в которых есть квартиры с видом на определенный объект ( например парк, реку, останкинскую телебашню, москва сити или другую достопримечательность).

Показать информацию о количестве квартир с таким видом их средние площади и стоимости.  

Вывести список жилых комплексов с наибольшей разницей между стоимостью квартир в них и среденей стоимостью квартир в районе. Рассчитать процентное отклонение цены каждой квартиры от средней цены по району.  

Начиная с 2023 года, построить несколько графиков по месяцам : стоимости кв метра квартир, курса доллара(сколько рублей за один доллар), ключевой ставки, уровень безработиц, численность населения в городе, среднее количество детей в семье, количество сделок по ипотеке, количество сделок по льготным ипотечным программам,  

Найти все новостройки информация о которых появилась в 2023 году имеют рейтинг выше среднего по городу. Вывести список этих новостроек, среднее, минимальное и максимальное количество этажей в них, в разрезе типа квартир ( тип это количество комнат ) с количеством квартир.  

Вывести весь жк квартиры в которых были введены в эксплуатацию с чистовой отделкой в 2024 году  

вывести список жк бизнес класса в ВАО города Москвы района Измайлово, ключи от которых владельцы получили в 2023  

Выведите жк где соотношение количества квартир к количеств парковочных мест меньше 2.5  

Введите количество квартир в кирпичных домах города Москвы  

Определите среднее время оживания лита в домах застройщика level  

Выведите список жк от которых до ближайшей станции метро 10 минут пешком  

Выведите общее число объектов застройщиков которые больше 20 лет на рынке, и при качестве объектов выше среднего.

# Задачи на подумать и предложить

Выдвинуть 10 гипотез и проверить их на основе данных ( если данных не хватает то добавить данными из интернета)  

Гипотеза 1 - стоимость квартиры зависит от площади  

Гипотеза 2 - стоимость квартиры зависит от расстояния до центра города  

Гипотеза 3 -  